In [1]:
ls

anomaly_detection_data_extraction.ipynb  labels_json/
data.yaml                                labels_normal/
images/                                  labels_phone/
images_normal/                           labels_sleep/
images_phone/                            runs/
images_sleep/                            trash/
labels/                                  txt_file/
labels.cache                             yolo_generate.ipynb
labels_anomaly/                          yolov8n.pt
labels_anomaly.tar


In [8]:
import json
from glob import glob
from tqdm import tqdm
import os 

#### yolo 학습에 사용될 BBox 및 Class 정의, txt 추출

In [107]:
file_list = os.listdir('./labels_json')
for i in tqdm(file_list):
    file_path = f'./labels_json/{i}'
      
    if not os.path.exists(file_path):
        continue

    with open(file_path) as json_file:
        json_data = json.load(json_file)

    # 파일 이름 및 BoundingBox 크기 추출    
    file_name = json_data.get('FileName').replace('.jpg', '')
    size = json_data.get('BoundingBox')
    
    # BoundingBox 정보를 좌표 및 크기로 변환하는 함수 정의
    def info(label):
        bounding_box = size.get(label)
        if bounding_box is not None:
            
            # BoundingBox가 이미지 크기를 초과하지 않도록 보정
            if float(bounding_box[0]) + float(bounding_box[2]) > 1280 :
                bounding_box[2] = (1280 - float(bounding_box[0]))
                
            if (bounding_box[1]) + (bounding_box[3]) > 720 :
                bounding_box[3] = (720 - float(bounding_box[1]))

            # BoundingBox 정보를 좌표 및 크기로 변환
            x = ((float(bounding_box[0]) + (float(bounding_box[2]) / 2)) / json_data.get('size')[0])
            y = ((float(bounding_box[1]) + (float(bounding_box[3]) / 2)) / json_data.get('size')[1])
            w = (float(bounding_box[2]) / json_data.get('size')[0])
            h = (float(bounding_box[3]) / json_data.get('size')[1])
            return x, y, w, h
    
        else:
            return None
        
    # body와 face의 BoundingBox 정보 추출              
    body_info = info('body')
    face_info = info('face')
    
    # 텍스트 파일에 BoundingBox 정보 작성
    with open(f'./labels/{file_name}.txt', 'a') as text_file:
        if body_info is not None:
            text_file.write(f"0 {' '.join(map(str, body_info))}\n")
        if face_info is not None:
            text_file.write(f"1 {' '.join(map(str, face_info))}\n")

100%|██████████| 237845/237845 [04:14<00:00, 935.00it/s] 


In [116]:
path = './labels'
lst = os.listdir(path)
img_lst = []

for i in tqdm(range(len(lst))):
    filename = lst[i]
    num = filename.split('.')[0]
    img_lst.append(num)

100%|██████████| 237845/237845 [00:00<00:00, 1315681.22it/s]


#### train_test_split 진행

In [118]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(img_lst, test_size = 0.2)

In [119]:
dir = '/home/work/VisionAI/monitoring/images/'
#전체 train test
for i in tqdm(train):
    with open(f'./txt_file/train.txt','a') as text_file:
        text_file.write(f'{dir}{i}.jpg\n')

100%|██████████| 190276/190276 [01:36<00:00, 1970.73it/s]


In [32]:
dir = '/home/work/VisionAI/monitoring/images/'
for i in tqdm(test):
    with open(f'./txt_file/test.txt','a') as text_file:
        text_file.write(f'{dir}{i}.jpg\n')

100%|██████████| 47555/47555 [00:24<00:00, 1959.20it/s]


In [33]:
from sklearn.model_selection import train_test_split
train1, val = train_test_split(train, test_size = 0.2)

In [34]:
#학습에 쓸 train
dir = '/home/work/VisionAI/monitoring/images/'
for i in tqdm(train1):
    with open(f'./txt_file/train1.txt','a') as text_file:
        text_file.write(f'{dir}{i}.jpg\n')

100%|██████████| 152174/152174 [01:17<00:00, 1957.71it/s]


In [35]:
dir = '/home/work/VisionAI/monitoring/images/'
for i in tqdm(val):
    with open(f'./txt_file/val.txt','a') as text_file:
        text_file.write(f'{dir}{i}.jpg\n')

100%|██████████| 38044/38044 [00:19<00:00, 1993.12it/s]
